In [1]:
function snoob(x)
    next = 0
    if(x>0)
        smallest = x & -(x)
        ripple = x + smallest
        ones = xor(x,ripple)
        ones = div((ones >> 2),smallest)
        next = ripple | ones
    end
    return next
end


snoob (generic function with 1 method)

In [2]:
function count_bit(n)
    count = 0
    while (n>0)
        count += n & 1
        n >>= 1
    end
    return count 
end


count_bit (generic function with 1 method)

In [3]:
function init_parameters(N,Sz)
    Nup = div(N,2) + Sz
    Nhilbert = binomial(N,Nup)
    ihfbit = 1 << (div(N,2))
    irght = ihfbit-1
    ilft = xor(((1<<N)-1),irght)
    iup = (1<<(N-Nup))-1
    return Nup, Nhilbert, ihfbit, irght, ilft, iup
end


init_parameters (generic function with 1 method)

In [4]:
function make_list(N,Nup,Nhilbert,ihfbit,irght,ilft,iup)
    list_1 = zeros(Int,Nhilbert)
    list_ja = zeros(Int,ihfbit)
    list_jb = zeros(Int,ihfbit)
    ii = iup
    ja = 0
    jb = 0
    ia_old = ii & irght
    ib_old = div((ii & ilft),ihfbit)
    list_1[1] = ii
    list_ja[ia_old+1] = ja
    list_jb[ib_old+1] = jb
    ii = snoob(ii)
    for i in 1:Nhilbert-1
        ia = ii & irght
        ib = div((ii & ilft),ihfbit)
        if ib == ib_old
            ja += 1
        else
            jb += ja+1
            ja = 0
        end
        list_1[i+1] = ii
        list_ja[ia+1] = ja
        list_jb[ib+1] = jb
        ia_old = ia
        ib_old = ib
        ii = snoob(ii)
    end
    return list_1, list_ja, list_jb
end


make_list (generic function with 1 method)

In [5]:
function get_ja_plus_jb(ii,irght,ilft,ihfbit,list_ja,list_jb)
    ia = ii & irght
    ib = div((ii & ilft),ihfbit)
    ja = list_ja[ia+1]
    jb = list_jb[ib+1]
    return ja+jb
end


get_ja_plus_jb (generic function with 1 method)

In [6]:
function make_hamiltonian(J1,D1,N,Nhilbert,irght,ilft,ihfbit,list_1,list_ja,list_jb)
    listki = ones(Int,(N+1)*Nhilbert)
    loc = ones(Int,(N+1)*Nhilbert)
    elemnt = zeros((N+1)*Nhilbert)
    for k in 0:N
        for i in 0:Nhilbert-1
            listki[k*Nhilbert+i+1] = i+1
        end
    end
    for k in 0:N-1
        isite1 = k
        isite2 = mod((k+1),N)
        is1 = 1<<isite1
        is2 = 1<<isite2
        is0 = is1 + is2
        wght = -2.0*J1[k+1]
        diag = wght*0.5*D1[k+1]
        for i in 0:Nhilbert-1
            ii = list_1[i+1]
            ibit = ii & is0
            if (ibit==0 || ibit==is0)
                elemnt[N*Nhilbert+i+1] -= diag
                loc[N*Nhilbert+i+1] = i+1
            else
                elemnt[N*Nhilbert+i+1] += diag
                loc[N*Nhilbert+i+1] = i+1
                iexchg = xor(ii,is0)
                newcfg = get_ja_plus_jb(iexchg,irght,ilft,ihfbit,list_ja,list_jb)
                elemnt[k*Nhilbert+i+1] = -wght
                loc[k*Nhilbert+i+1] = newcfg+1
            end
        end
    end
    HamCSC = dropzeros(sparse(listki,loc,elemnt))
    return HamCSC
end


make_hamiltonian (generic function with 1 method)

In [7]:
using LinearAlgebra
using Arpack
using SparseArrays

N = 16 # should be N>=4
Sz = 0

Nup, Nhilbert, ihfbit, irght, ilft, iup = init_parameters(N,Sz)

binirght = string(irght,base=2,pad=N)
binilft = string(ilft,base=2,pad=N)
biniup = string(iup,base=2,pad=N)

println("N=",N)
println("Sz=",Sz)
println("Nup=",Nup)
println("Nhilbert=",Nhilbert)
println("ihfbit=",ihfbit)
println("irght,binirght=",irght," ",binirght)
println("ilft,binilft=",ilft," ",binilft)
println("iup,biniup=",iup," ",biniup)

@time list_1, list_ja, list_jb = make_list(N,Nup,Nhilbert,ihfbit,irght,ilft,iup)

#println("list_1=",list_1)
#println("list_ja=",list_ja)
#println("list_jb=",list_jb)

#println("")
#println("i ii binii ja+jb")
#for i in 0:Nhilbert-1
#    ii = list_1[i+1]
#    binii = string(ii,base=2,pad=N)
#    ind = get_ja_plus_jb(ii,irght,ilft,ihfbit,list_ja,list_jb)
#    println(i," ",ii," ",binii," ",ind)
#end


N=16
Sz=0
Nup=8
Nhilbert=12870
ihfbit=256
irght,binirght=255 0000000011111111
ilft,binilft=65280 1111111100000000
iup,biniup=255 0000000011111111
  0.045965 seconds (39.98 k allocations: 2.176 MiB)


([255, 383, 447, 479, 495, 503, 507, 509, 510, 639  …  64896, 65025, 65026, 65028, 65032, 65040, 65056, 65088, 65152, 65280], [0, 0, 1, 0, 2, 1, 2, 0, 3, 3  …  24, 4, 55, 25, 26, 5, 27, 6, 7, 0], [0, 1, 9, 17, 45, 53, 81, 109, 165, 173  …  12669, 12697, 12705, 12761, 12789, 12817, 12825, 12853, 12861, 12869])

In [8]:
J1 = ones(N) # J_{ij}>0: AF
D1 = ones(N) # D_{ij}>0: AF

@time Ham = make_hamiltonian(J1,D1,N,Nhilbert,irght,ilft,ihfbit,list_1,list_ja,list_jb)
#println(Ham)

@time ene,vec = eigs(Ham, nev=5, which=:LM)
#println("# GS energy:",ene[1])
println("# energy:",ene[1]," ",ene[2]," ",ene[3]," ",ene[4]," ",ene[5])
#vec_sgn = sign(argmax(vec[:,1]))
#println("# GS wave function:")
#for i in 0:Nhilbert-1
#    ii = list_1[i+1]
#    binii = string(ii,base=2,pad=N)
#    println(i," ",vec[i+1,1]*vec_sgn," ",binii)
#end


  0.212200 seconds (463.30 k allocations: 35.633 MiB, 2.44% gc time)
  2.454857 seconds (7.13 M allocations: 350.538 MiB, 8.14% gc time)
# energy:-28.569185442467138 -27.48842671346589 -26.78618970637527 -26.093628229524846 -26.09362822952474
